# Initialization

In [ ]:
import os

if os.getcwd().endswith("notebooks"):
    os.chdir("..")
    print("using project root as working dir")

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%load_ext tensorboard
%tensorboard --logdir runs

In [ ]:
import torch

device = "cuda" if torch.cuda.is_available() else ("mps" if torch.backends.mps.is_available() else "cpu")
print(f"using {device} device")

# Run

In [ ]:
from src.graph import random_geometric_graph, random_graph
from dataclasses import asdict
from src.args import gridsearch_args, Args
from datetime import datetime
import time
import seaborn as sns
import pandas as pd

from src.evaluator import Evaluator

experiment = dict(
    key=datetime.now().strftime("%d-%m--%H-%M"),
    reps=3,
    args=gridsearch_args(),
)
df_result = pd.DataFrame({})

run_len = len(experiment['args']) * experiment['reps']
for i, args in enumerate(experiment["args"]):
    # override args for specific test
    # args.graph_type = "random"
    # args.early_stopping = False
    # args.epochs = 150

    for rep in range(experiment["reps"]):
        run_index = i * experiment['reps'] + rep
        print(f"running evaluator {run_index}/{run_len}")

        if args.graph_type == "rgg":
            graph = random_geometric_graph(
                size=args.graph_size,
                radius=args.rgg_radius
            )
        elif args.graph_type == "random":
            graph = random_graph(
                size=args.graph_size
            )
        else:
            raise ValueError(f"invalid graph type: {args.graph_type}")

        evaluator = Evaluator(
            graph=graph,
            args=args,
            writer_log_dir=f"runs/{experiment['key']}/s{args.epoch_graph_size}-a{args.epoch_graph_alpha}-b{args.epoch_graph_boredom_pth}-l{args.layers}xs{args.layer_size}--{rep}",
            device=device
        )
        start_time = time.time()
        evaluator.train(
            optimizer=torch.optim.Adam(evaluator.net.parameters(), lr=1e-3),
            save_fig=False
        )
        end_time = time.time()
        test_loss, test_ap, test_f1 = evaluator.test(
            epoch=args.epochs,
            save_fig=True
        )

        df_result = pd.concat([
            df_result,
            pd.Series({
                # run meta
                "run_index": run_index,
                "run_time": end_time - start_time,
                # run results
                "loss": test_loss,
                "ap": test_ap,
                "f1": test_f1,
                # run args
                **asdict(args),
            }).to_frame().T
        ], ignore_index=True)

    # save results just in case the run fails mid run
    df_result.to_csv(f"{experiment['key']}.csv.zip", index=False, compression=dict(method='zip', archive_name=f"{experiment['key']}.csv"))

# Results

In [ ]:
# mean score (bars)
df_result_mean_ap = df_result.groupby(["epoch_graph_size", "epoch_graph_alpha"], as_index=False)["ap"].mean().sort_values(by=['ap'])
print(df_result_mean_ap)

sns.catplot(
    data=df_result_mean_ap,
    col="epoch_graph_size",
    x="epoch_graph_alpha",
    y="ap",
    palette="Spectral",
    kind = "bar"
)

In [ ]:
# score per epoch graph size and alpha (line with std)
sns.relplot(
    data=df_result, kind="line",
    hue="epoch_graph_size",
    x="epoch_graph_alpha",
    y="ap",
    errorbar="sd",
    palette="Spectral",
    aspect=2,
)

In [ ]:
# run time per graph and epoch graph size (line with std)
sns.relplot(
    data=df_result, kind="line",
    hue="epoch_graph_size",
    x="graph_size",
    y="run_time",
    errorbar="sd",
    palette="Spectral",
    aspect=2,
)

# Next Tasks

- bigger graphs
    - how much more time?
- different graph types
    - girgs (https://github.com/chistopher/girgs) is like power law
- rgg: reconstruct threshold radius based on test nodes and f1-score

# Writing
- best 10-20 pages
- until 18.